In [109]:
import pandas as pd 
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GroupKFold, BaseCrossValidator, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

We start importing the data set to study the different columns relevance.

In [110]:
raw_data = pd.read_csv("dataset.csv")
raw_data

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896,140685,Female,27.0,Surat,Student,5.0,0.0,5.75,5.0,0.0,5-6 hours,Unhealthy,Class 12,Yes,7.0,1.0,Yes,0
27897,140686,Male,27.0,Ludhiana,Student,2.0,0.0,9.40,3.0,0.0,Less than 5 hours,Healthy,MSc,No,0.0,3.0,Yes,0
27898,140689,Male,31.0,Faridabad,Student,3.0,0.0,6.61,4.0,0.0,5-6 hours,Unhealthy,MD,No,12.0,2.0,No,0
27899,140690,Female,18.0,Ludhiana,Student,5.0,0.0,6.88,2.0,0.0,Less than 5 hours,Healthy,Class 12,Yes,10.0,5.0,No,1


# Relevance of all variables: 

id: Unique identifier for each respondent; irrelevant for analysis (can be dropped).

Gender: Helps identify gender-based trends or disparities in depression levels.

Age: Relevant for analyzing how depression correlates with age.

City: Useful for identifying regional patterns in mental health.

Profession: Captures the impact of professional status (e.g., being a student) on depression.

Academic Pressure: Key to understanding the role of academic stress in mental health.

Work Pressure: Helps analyze how work-related stress impacts depression.

CGPA: Reflects academic performance, which may correlate with mental health.

Study Satisfaction: Indicates how satisfaction with studies influences depression.

Job Satisfaction: Relevant for understanding the impact of job fulfillment on mental well-being.

Sleep Duration: Critical for exploring the relationship between sleep habits and mental health.

Dietary Habits: Highlights how diet quality affects mental health.

Degree: Useful for analyzing mental health trends across different education levels.

Have you ever had suicidal thoughts?: Directly related to understanding extreme mental health risks.

Work/Study Hours: Reflects the impact of workload on mental health.

Financial Stress: Important for evaluating the role of financial factors in mental health.

Family History of Mental Illness: Crucial for understanding genetic or familial influences on depression.

Depression: The target variable for predictive analysis.

# Handling Missing Values

In [111]:
# Delete id column
raw_data.drop(columns=['id'], axis=1, inplace=True)

# Check missing values
missing_values = raw_data.isnull().sum()
missing_values

Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         3
Family History of Mental Illness         0
Depression                               0
dtype: int64

We see that the missing values exclusively are in the financial Stress column. We replace these 3 missing values by median of the whole datasets Financial Stress values.

In [112]:
financial_stress_median = raw_data['Financial Stress'].median()
raw_data['Financial Stress'].fillna(financial_stress_median, inplace=True)

/tmp/ipykernel_189523/2780200284.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  raw_data['Financial Stress'].fillna(financial_stress_median, inplace=True)


Since it's a survey, we ensure that all the "yes" and "no" are standardized, are typed in lowercase in our case.

In [113]:
raw_data['Have you ever had suicidal thoughts ?'] = raw_data['Have you ever had suicidal thoughts ?'].str.strip().str.lower()
raw_data['Family History of Mental Illness'] = raw_data['Family History of Mental Illness'].str.strip().str.lower()


Converting categorical Sleep Duration values (e.g., "5-6 hours", "Less than 5 hours") into numerical representations based on estimated averages.

In [114]:
def transform_sleep_duration(duration):
    if "less than 5" in duration.lower():
        return 4.5
    elif "5-6" in duration.lower():
        return 5.5
    elif "7-8" in duration.lower():
        return 7.5
    elif "more than 8" in duration.lower():
        return 9.0
    else:
        return None 

raw_data['Sleep Duration'] = raw_data['Sleep Duration'].apply(transform_sleep_duration)

In [115]:
raw_data

,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5.5,Healthy,B.Pharm,yes,3.0,1.0,no,1
1,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5.5,Moderate,BSc,no,3.0,2.0,yes,0
2,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,4.5,Healthy,BA,no,9.0,1.0,yes,0
3,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7.5,Moderate,BCA,yes,4.0,5.0,yes,1
4,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5.5,Moderate,M.Tech,yes,1.0,1.0,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896,Female,27.0,Surat,Student,5.0,0.0,5.75,5.0,0.0,5.5,Unhealthy,Class 12,yes,7.0,1.0,yes,0
27897,Male,27.0,Ludhiana,Student,2.0,0.0,9.40,3.0,0.0,4.5,Healthy,MSc,no,0.0,3.0,yes,0
27898,Male,31.0,Faridabad,Student,3.0,0.0,6.61,4.0,0.0,5.5,Unhealthy,MD,no,12.0,2.0,no,0
27899,Female,18.0,Ludhiana,Student,5.0,0.0,6.88,2.0,0.0,4.5,Healthy,Class 12,yes,10.0,5.0,no,1


Categorical variables (e.g., "Gender", "City") contain values that are textual or non-numeric (e.g., "Male", "Female"). Most machine learning models require numeric input, so these variables need to be converted into a numerical format. One-hot encoding is one method to achieve this.

In [116]:
categorical_columns = ['Gender', 'City', 'Profession', 'Dietary Habits', 'Degree']
data = pd.get_dummies(raw_data, columns=categorical_columns, drop_first=True)

The target variable Depression has values 0 and 1, representing "No" and "Yes" states, respectively. Converting it into a categorical type makes it explicitly clear that this is not a continuous variable but rather a binary classification

In [117]:
data['Depression'] = data['Depression'].astype('category')

# Logistic regression

Let's try to strat with a logistic regression in order to predict depression cases. This choice is highly motivated by the interpretability of the approach.

In [118]:
# Convert target variable to numeric
y = data['Depression'].map({0: 0, 1: 1})  # Assuming binary

# Drop target column
X = data.drop(columns=['Depression'])

X['Have you ever had suicidal thoughts ?'] = X['Have you ever had suicidal thoughts ?'].map({'yes': 1, 'no': 0})
X['Family History of Mental Illness'] = X['Family History of Mental Illness'].map({'yes': 1, 'no': 0})

# Handle missing values
X.fillna(X.median(), inplace=True)  # Replace missing numerical values with median
X = pd.get_dummies(X, drop_first=True)  # Convert categorical variables to numeric

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def asses_model(estimator:ClassifierMixin): 
    """
    Function to fit, predict and asses model.
    """
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    scores = accuracy_score(y_test, y_pred)

    print(f"Accuracy: {scores}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

def model_selection(estimator:ClassifierMixin, cv:BaseCrossValidator|None = None, param_grid:dict = {}):
    """
    Function to help for model selection with cross-validation like methods and grid search
    """
    grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, 
                            cv=cv, n_jobs=-1, verbose=0, scoring='accuracy')
    
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    #print(grid_search.cv_results_)
    print("best parameters : ", grid_search.best_params_)

    return best_model


In [25]:


# Initialize the logistic regression model
logistic_model = LogisticRegression(max_iter=100, random_state=42, class_weight='balanced', verbose=0)
param_grid = {
    'penalty': ['l2', 'elasticnet', 'none', "l1"],
    'C': [0.1, 1, 10, 100],
    'max_iter': [100, 200, 300],
    'l1_ratio': [None, 0.1, 0.5, 0.9],  # ratio for combinaision of l1 and l2 in elasticnet
}
grid_logistic_model = model_selection(logistic_model, param_grid=param_grid)
asses_model(grid_logistic_model)


/home/ndubourg/Documents/Cours/Master2/supervised_learning/Projet/venv/lib64/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ndubourg/Documents/Cours/Master2/supervised_learning/Projet/venv/lib64/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for a

best parameters :  {'C': 0.1, 'l1_ratio': None, 'max_iter': 300, 'penalty': 'l2'}
Accuracy: 0.8317505823329152

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.82      0.80      2343
           1       0.87      0.84      0.85      3238

    accuracy                           0.83      5581
   macro avg       0.83      0.83      0.83      5581
weighted avg       0.83      0.83      0.83      5581



The classification report indicates that the logistic regression model is performing reasonably well, achieving an overall accuracy of 79.79%. For the "Not Depressed" class (0), the model has a precision of 0.74, meaning 74% of the predictions for this class are correct. The recall for this class is 0.79, indicating that the model identifies 79% of the actual "Not Depressed" cases. The F1-score for this class is 0.77, showing a good balance between precision and recall.

For the "Depressed" class (1), the model performs slightly better, with a precision of 0.84, meaning 84% of predictions for this class are accurate. The recall is 0.80, showing that the model successfully identifies 80% of actual depression cases. The F1-score for this class is 0.82, reflecting a strong trade-off between precision and recall.

Overall, the macro and weighted averages for precision, recall, and F1-score are all approximately 0.80, suggesting that the model balances both classes effectively. These results indicate an improvement compared to the previous model, likely due to better handling of class imbalance or feature selection. However, further optimization or exploration of alternative models could improve the performance further, particularly in identifying the "Not Depressed" class more accurately.

# try random forest

In [26]:
random_forest_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50],  # Number of trees
    'max_depth': [10, 20, 30],
    'min_samples_split': [5, 10],  # Nombre minimal d'échantillons nécessaires pour diviser un nœud
    'min_samples_leaf': [2, 4],  # Nombre minimal d'échantillons nécessaires dans un nœud terminal
    'bootstrap': [True],
    'criterion': ['gini'],
    'max_samples': [None, 0.5, 0.8],  # if bootstrap = true
    'oob_score': [True],  # Out of bag error
}

grid_random_forest = model_selection(random_forest_model, param_grid=param_grid)
asses_model(random_forest_model)

best parameters :  {'bootstrap': True, 'criterion': 'gini', 'max_depth': 30, 'max_samples': 0.5, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 50, 'oob_score': True}
Accuracy: 0.8301379681060742

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.78      0.79      2343
           1       0.84      0.87      0.86      3238

    accuracy                           0.83      5581
   macro avg       0.83      0.82      0.82      5581
weighted avg       0.83      0.83      0.83      5581



In [27]:
# SVC_model = SVC()

# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'kernel': ['linear'],  # Kernels , 'rbf', 'poly', 'sigmoid'
#     'degree': [2],  # for polynomial kernel
#     'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
#     'coef0': [0.0, 0.1, 0.5],  # for poly and sigmoid
#     'tol': [1e-3, 1e-4],  # tolerance to stop algo
# }

# grid_SVC = model_selection(SVC_model, param_grid=param_grid)
# asses_model(SVC_model)

In [29]:
knn_model = KNeighborsClassifier()

knn_param_grid = {
    'n_neighbors': [10, 15, 20],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan'],
}

grid_knn = model_selection(knn_model, param_grid=knn_param_grid)
asses_model(knn_model)

best parameters :  {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 20, 'weights': 'distance'}
Accuracy: 0.7848055903959864

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.70      0.73      2343
           1       0.79      0.85      0.82      3238

    accuracy                           0.78      5581
   macro avg       0.78      0.77      0.78      5581
weighted avg       0.78      0.78      0.78      5581



In [28]:
kf = KFold(n_splits=10)
kk = GroupKFold(10)
type(kf)
type(kk)

sklearn.model_selection._split.GroupKFold

# Deep learning

We don't obtain a good enouth results let's try deep learning.

In [119]:
import torch
import torch.nn as nn
import torch.optim as optim

In [127]:
# 1. Définir le modèle de réseau de neurones
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_hidden_layers=8):
        super(SimpleNN, self).__init__()
        
        # Couche d'entrée
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        
        # Couches cachées (répétées)
        self.hidden_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.ReLU()
            ) for _ in range(num_hidden_layers)
        ])
        
        # Couche de sortie
        self.fc_out = nn.Linear(hidden_size, 1)  # Sortie avec 1 neurone pour la classification binaire
        self.sigmoid = nn.Sigmoid()  # Fonction d'activation Sigmoid pour la sortie

    def forward(self, x):
        # Passer dans la couche d'entrée
        x = self.fc1(x)
        
        # Passer dans les couches cachées
        for layer in self.hidden_layers:
            x = layer(x)
        
        # Passer dans la couche de sortie
        x = self.fc_out(x)
        x = self.sigmoid(x)  # Appliquer la fonction Sigmoid pour une sortie entre 0 et 1
        
        return x


In [124]:
# Pandas dataframe to tensor
X_train_float = X_train.astype('float32')
X_test_float = X_test.astype('float32')
y_train_float = y_train.astype('float32')
y_test_float = y_test.astype('float32')

X_train_tensor = torch.tensor(X_train_float.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_float.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_float.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_float.values, dtype=torch.float32).unsqueeze(1)

print(X_train_tensor.size())
print(y_train_tensor.size())

torch.Size([22320, 106])
torch.Size([22320, 1])


In [128]:
# Parameters
input_size = 106
hidden_size = 32
learning_rate = 0.001
epochs = 10000

# Loss function and optimizer selection
model = SimpleNN(input_size, hidden_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train model
for epoch in range(epochs):
    optimizer.zero_grad() # Reinitialize gradient
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor) # Compute loss

    # Compute gradient and update coefficients
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [100/10000], Loss: 0.5508
Epoch [200/10000], Loss: 0.5420
Epoch [300/10000], Loss: 0.5403
Epoch [400/10000], Loss: 0.5392
Epoch [500/10000], Loss: 0.5388
Epoch [600/10000], Loss: 0.5396
Epoch [700/10000], Loss: 0.5372
Epoch [800/10000], Loss: 0.5368
Epoch [900/10000], Loss: 0.5366
Epoch [1000/10000], Loss: 0.5510
Epoch [1100/10000], Loss: 0.5362
Epoch [1200/10000], Loss: 0.5361
Epoch [1300/10000], Loss: 0.5386
Epoch [1400/10000], Loss: 0.5357
Epoch [1500/10000], Loss: 0.5354
Epoch [1600/10000], Loss: 0.5450
Epoch [1700/10000], Loss: 0.5408
Epoch [1800/10000], Loss: 0.5358
Epoch [1900/10000], Loss: 0.5361
Epoch [2000/10000], Loss: 0.5397
Epoch [2100/10000], Loss: 0.5376
Epoch [2200/10000], Loss: 0.5344
Epoch [2300/10000], Loss: 0.5339
Epoch [2400/10000], Loss: 0.5341
Epoch [2500/10000], Loss: 0.5342
Epoch [2600/10000], Loss: 0.5352
Epoch [2700/10000], Loss: 0.5340
Epoch [2800/10000], Loss: 0.5345
Epoch [2900/10000], Loss: 0.5363
Epoch [3000/10000], Loss: 0.5345
Epoch [3100/10000],

### Evaluation Metrics

**1. Recall (Sensitivity, True Positive Rate):**

$$
\text{Recall} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Negatives (FN)}}
$$

**2. Precision (Positive Predictive Value):**

$$
\text{Precision} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Positives (FP)}}
$$

**3. F1 Score (Harmonic Mean of Precision and Recall):**

$$
\text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$


In [129]:

# Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation for evaluation
    predictions = model(X_test_tensor)  # Model output (logits)

# Apply threshold to get binary class predictions
predicted_classes = (predictions > 0.5).float()

# Convert tensors to NumPy arrays for compatibility with sklearn
y_true = y_test_tensor.numpy()
y_pred = predicted_classes.numpy()

# Generate the confusion matrix and other metrics
conf_matrix = confusion_matrix(y_true, y_pred)
precision = conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[0][1])
recall = conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])
F1_score = 2 * (precision * recall / (precision + recall))
accuracy = (conf_matrix[1][1] + conf_matrix[0][0]) / sum(sum(conf_matrix))

print("Confusion Matrix:")
print(conf_matrix)
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1-score: {F1_score}")
print(f"Accuracy: {accuracy}")


Confusion Matrix:
[[1941  402]
 [ 549 2689]]
Recall: 0.8304508956145769
Precision: 0.8699450016175995
F1-score: 0.8497392953073154
Accuracy: 0.8296004300304605
